# Exercises Week 11-12: Graded exercise session (part A)

**Course**: [Life Sciences engineering: genome to function](https://go.epfl.ch/BIO-411) (BIO-411)

**Professors**:  _Gönczy Pierre_, _Naef Felix_, _McCabe Brian Donal_

SSV, MA, 2023-2024


In [ ]:
#import librairies
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson, nbinom
from scipy.integrate import odeint
from scipy.special import gammaln, digamma, polygamma

**Provide answers in the form of code, figures (if relevant) and short descriptions (in markdown cells) in those notebooks. Submit your notebook to Moodle, please make sure to execute every cell.**

## Exercise 1: Transcriptional bursting

Let's consider the transcriptional process in which an mRNA $x$ is produced at a rate $s$ and degraded at a rate $k$. In addition to the simple birth-death process that we simulated in week 10, we will now consider a two-state model for the promoter (Random Telegraph process). This means that the promoter can switch from an active ($g=1$) to inactive state ($g=0$) and vice-versa with respective rates $k_{off}$ and $k_{on}$.


The Telegraph process has the following elementary reactions:  
\begin{array}{lll}
(1, n) \longrightarrow (1, n+1) \textrm{ with rate } s \\
(g, n) \longrightarrow (g, n-1) \textrm{ with rate } kn \\
(1, n) \longrightarrow (0, n) \textrm{ with rate } k_{off} \\
(0, n) \longrightarrow (1, n) \textrm{ with rate } k_{on}
\end{array}

We describe the states $(g, n)$ with $g\in \{1, 0\}$, $n \in \mathbb{N}$.

#### Question 1
1. Using the code template provided, which includes initial values and parameters, modify the Gillespie algorithm to simulate the Telegraph model. Focus on integrating the two-state model for the promoter accurately into the algorithm. You can refer to the Gillespie function from *Exercise week 10* as a starting point.

2. Systematically vary the rates (e.g increase or decrease $k$, $s$, $k_{on}$, $k_{off}$), plot representative traces and comment. Find cases that show qualitatively different behaviors.

#### Gillespie template

In [ ]:
def run_gillespie_telegraph(n0, g0, parameters, t_end, DT):
    """
    Simulates the Random telepgrah process using the Gillespie algorithm.

    Parameters:
    n0 : Initial count of species n.
    g0 : Initial promoter state g (0 or 1).
    parameters: Tuple containing transcription rate s, degradation rate k, switching rates k_on and k_off
    t_end : Total simulation time.
    DT: time interval for returning/printing the output

    Returns:
    n_print : Trajectory of species n counts over time.
    g_print : Trajectory of promoter state g over time.
    t_print : Time points for each recorded reaction.
    """
    # Initialize state variables and time
    n = n0   
    g = g0   
    t = 0    
    tp = 0  

    # Initialize output lists
    n_print=[]  
    g_print=[]  
    t_print=[]  

    # Unpack parameters
    s, k, k_on, k_off = parameters 

    while t < t_end: 
        
        # Define state change matrix S and rate vector depending on promoter state
        if g == 1: # Promoter is active 
            S = ... # FILL HERE
            rates = ... # FILL HERE
        else: # Promoter is inactive
            S = ... # FILL HERE
            rates = ... # FILL HERE

        # Calculate total rate and sample time to next reaction
        rates_tot = sum(rates)
        dt = np.random.exponential(scale=1.0 / rates_tot)
        t += dt

        # Determine which reaction occurs next
        rand = np.random.random()        
        rates_cumsum = np.cumsum(rates)/rates_tot

        i=0
        while rand >= rates_cumsum[i]:
            i=i+1

        # Update species n counts and promoter state g
        chosen_react=i
        g += S[chosen_react,0]
        n += S[chosen_react,1]
 
        # Record output at specified DT intervals (equally spaced output)
        while tp < t and t <= t_end:
            n_print.append(n)
            g_print.append(g)
            t_print.append(tp)
            tp += DT

    return n_print, g_print, t_print

In [ ]:
def plot_gillespie_telegraph(n, g, t):
    """
    Plots the Random telepgrah process using the Gillespie algorithm.

    Parameters:
    n: Species n counts over time.
    g: Promoter state g over time.
    t: Time points for each recorded reaction.

    """

    fig, axs = plt.subplots(2, figsize=(15, 5), sharex=True)

    # Plotting the number of molecules n over time
    axs[0].plot(t, n, label = 'n(t)', color='darkblue')
    axs[0].set_ylabel('# molecules n')

    # Plotting the promoter activity over time
    axs[1].vlines(x=t, ymin=0, ymax=np.array(g), colors='teal', lw=2, label='promoter activity')
    axs[1].set_ylim(0, 1.1)  # Set y-limits to make the promoter states clear (0 and 1)
    axs[1].set_ylabel('Promoter State')
    
    # Common x-label
    plt.xlabel('time')
    
    plt.show()

In [ ]:
#initial conditions for the state (g,n) at time t=0
g_0 = 1 # Initial promoter state
n_0 = 20 # Initial count of species n

DT = 0.01 # Time interval for returning/printing the output
t_end = 1000 #Total simulation time in minutes

#Parameters
s = 20 # Number of initiations per minute
k = 0.02 # Degradation rate, 1/k = 50 min
k_off = 1/15 #Switching off rate, t_on = 1/k_off = 15 min
k_on  = 1/120 #Switching on rate, t_off = 1/k_on = 120 min

parameters = [s, k, k_on, k_off]

# Run Gillespie simulation
n, g, t = run_gillespie_telegraph(n_0, g_0, parameters, t_end, DT)

# Plot the results
plot_gillespie_telegraph(n, g, t)


#### Question 2
1. Study the distribution of mRNA numbers generated by your simulation. Identify a parameter regime that results in a Poisson distribution of mRNA numbers. Show the empiral histogram and overlay an exact Poisson distribution and comment.

2. Find parameters where the mRNA distribution deviates from the Poisson (e.g. a Negative Binomial distribution). Discuss why these changes lead to a different distribution. 

3. For parameters that yield an approximate Poisson distribution, discuss how the parameters of the Telegraph model relate to the parameters of the Poisson distributions. 

4. Show a set of parameters that leads to a bimodal distribution of mRNA numbers.  


####  Helper functions for the Poisson and Negative Binomal distributions

In [ ]:
# Poisson distribution
def Poisson(lam, counts):
    """
    Calculate the Poisson distribution probability for given counts.
    
    Parameters:
    lam: The average number of events (rate parameter).
    counts: Observed counts.

    Returns:
    np.array: Probabilities corresponding to the observed counts.

    Notes:
    - This function uses the log-form of the Poisson probability mass function.
    - The gammaln function is used for calculating the logarithm of the factorial term.

    """

    logp = counts * np.log(lam) - lam - gammaln(counts + 1)
    return np.exp(logp)

# Negative Binomial distribution
# This may not be needed for the exercise, but it is included for completeness
def convert_params(mu, theta):
    """

    Convert mean and dispersion parameters to the shape and probability parameters of the negative binomial distribution.
    
    Parameters:
    mu: Mean of the distribution.
    theta: Dispersion parameter.

    Returns:
    tuple: Shape parameter (r) and success probability (p) of the negative binomial distribution.

    Notes
    - This conversion is necessary because different parameterizations are used in different contexts.

    """
    r = theta
    var = mu + 1 / r * mu ** 2
    p = (var - mu) / var
    return r, 1 - p

def pmf_nb(counts, mu, theta):
    """
    Compute the probability mass function of the negative binomial distribution for given counts.

    Parameters:
    counts: Observed counts.
    mu: Mean of the distribution.
    theta: Dispersion parameter.

    Returns:
    np.array: Probabilities corresponding to the observed counts.
    """


    return nbinom.pmf(counts, *convert_params(mu, theta))